In [ ]:
import psycopg2
from psycopg2 import sql
from concurrent.futures import ThreadPoolExecutor
import requests
from bs4 import BeautifulSoup
import re

conn = psycopg2.connect(
    dbname="PcBuild",
    user="postgres",
    password="123",
    host="localhost",
    port="5432"
)
cursor = conn.cursor()
def upsert_store(cursor, store_name):
    cursor.execute('SELECT "Id" FROM "Stores" WHERE "Name" = %s', (store_name,))
    store_id = cursor.fetchone()
    if store_id:
        return store_id[0]
    insert_query = """
        INSERT INTO "Stores" ("Name")
        VALUES (%s)
        RETURNING "Id";
    """
    cursor.execute(insert_query, (store_name,))
    store_id = cursor.fetchone()
    
    # Return the ID of the newly inserted store
    return store_id[0] if store_id else None


def fetch_page(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        return BeautifulSoup(response.content, 'html.parser')
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return None

def upsert_product(store_name, product_data):
    try:
        query = """
            SELECT "Id" FROM "Stores" WHERE "Name" = %s;
        """
        cursor.execute(query, (store_name,))
        store_id = cursor.fetchone()
        
        if not store_id:
            print(f"Error: Store {store_name} not found.")
            return  
        store_id = store_id[0]
        cursor.execute(
            """
            SELECT "Id", "CurrentPrice" FROM "Products"
            WHERE "ProductUrl" = %s AND "StoreId" = %s;
            """,
            (product_data['link'], store_id)
        )
        product = cursor.fetchone()

        if product:
            product_id, current_price = product
            if abs(float(current_price) - float(product_data['price'])) > 0.01: 
                cursor.execute(
                    """
                    UPDATE "Products"
                    SET "CurrentPrice" = %s
                    WHERE "Id" = %s;
                    """,
                    (product_data['price'], product_id)
                )
                cursor.execute(
                    """
                    INSERT INTO "Prices" ("Value", "ProductId", "Date")
                    VALUES (%s, %s, NOW());
                    """,
                    (product_data['price'], product_id)
                )
        else:
            print(f"Inserting product: {product_data['title']} - {product_data['price']}")

            cursor.execute(
                """
                INSERT INTO "Products" ("Name", "CurrentPrice", "StoreId", "ImageUrl", "ProductUrl")
                VALUES (%s, %s, %s, %s, %s)
                RETURNING "Id";
                """,
                (
                    product_data['title'],
                    product_data['price'],
                    store_id,
                    product_data['image_url'],
                    product_data['link']
                )
            )
            product_id = cursor.fetchone()[0]
            cursor.execute(
                """
                INSERT INTO "Prices" ("Value", "ProductId", "Date")
                VALUES (%s, %s, NOW());
                """,
                (product_data['price'], product_id)
            )
        conn.commit()
    except Exception as e:
        print(f"Error upserting product: {e}")
        conn.rollback()


# Function to scrape Sigma website
def scrape_sigma():
    base_url = "https://www.sigma-computer.com"
    main_page_url = f"{base_url}/home"
    main_soup = fetch_page(main_page_url)
    if not main_soup:
        return

    category_links = main_soup.select('div.sub-menu a.main-menu')
    for category_link in category_links:
        category_name = category_link.text.strip().lower()
        category_url = base_url + "/" + category_link['href']
        current_page_url = category_url + "&page=1"

        while True:
            soup = fetch_page(current_page_url)
            if not soup:
                break

            containers = soup.find_all('div', class_='product-item-container')
            for container in containers:
                anchor = container.find('a', href=True, title=True)
                title = anchor['title'] if anchor else "No title found"
                link = base_url + "/" + anchor['href'] if anchor else "No link found"
                price_span = container.find('span', class_='price-new')
                image_tag = container.find('img')
                image_url = base_url + "/" + image_tag['src'] if image_tag else "No image found"
                price = re.sub(r'[^\d.]', '', price_span.text.strip()) if price_span else None
                store_name = "Sigma Computer"

                if price:
                    product_data = {
                        "title": title,
                        "price": price,
                        "link": link,
                        "image_url": image_url,
                        "category": category_name
                    }
                    upsert_product(store_name, product_data)

            pagination = soup.find('ul', class_='pagination')
            next_page = pagination.find('a', rel='next') if pagination else None
            if next_page:
                current_page_url = next_page['href']
            else:
                break

def scrape_elnour():
    base_url = "https://elnour-tech.com/en/"
    main_soup = fetch_page(base_url)
    if not main_soup:
        return

    category_links = main_soup.select('.menu-item.menu-item-type-taxonomy.menu-item-object-product_cat')
    for category_link in category_links:
        category_name = category_link.text.strip().lower()
        category_url = category_link.select_one('a').get('href') if category_link.select_one('a') else None
        if not category_url:
            continue

        page_num = 1
        while True:
            page_url = f"{category_url}/page/{page_num}"
            soup = fetch_page(page_url)
            if not soup:
                break

            products = soup.select('.wd-product.product-grid-item')
            for product in products:
                add_to_cart_btn = product.select_one('.add_to_cart_button')
                if add_to_cart_btn:
                    title = product.select_one('.wd-entities-title').text.strip() if product.select_one('.wd-entities-title') else None
                    price_text = product.select_one('bdi').text.strip() if product.select_one('bdi') else None
                    price = re.sub(r'[^\d.]', '', price_text) if price_text else None
                    link = product.select_one('.wd-entities-title a').get('href') if product.select_one('.wd-entities-title a') else None
                    img_url = product.select_one('.product-wrapper img').get('src') if product.select_one('.product-wrapper img') else None
                    store_name = "Elnour Tech"

                    if price:
                        product_data = {
                            "title": title,
                            "price": price,
                            "link": link,
                            "image_url": img_url,
                            "category": category_name
                        }
                        upsert_product(store_name, product_data)

            next_page = soup.select_one('a.page-numbers.next')
            if next_page:
                page_num += 1
            else:
                break
def scrape_and_store(store_names):
    with conn:
        with conn.cursor() as cursor:
            print("Upserting stores...")
            store_ids = {name: upsert_store(cursor, name) for name in store_names}
            conn.commit()

def main():
    websites = {
        "sigma": scrape_sigma,
        "elnour": scrape_elnour
    }
    stores = ["Sigma Computer", "Elnour Tech"]
    scrape_and_store(stores)
    with ThreadPoolExecutor() as executor:
        future_to_website = {executor.submit(scraper): name for name, scraper in websites.items()}
        for future in future_to_website:
            website = future_to_website[future]
            try:
                future.result()
            except Exception as e:
                print(f"Error scraping {website}: {e}")

if __name__ == "__main__":
    main()

conn.close()


Upserting stores...
Inserting product: ASUS PRIME H610M-A DDR5  LGA 1700 mATX motherboard with DDR5 PCIe 4.0 dual M.2 slots  Intel 1Gb Ethernet  WIFI 5  DisplayPort HDMI  VGA  USB 3.2 Gen 2 ports  Addressable Gen 2 headers - 4849
Inserting product: ASUS PRIME H610M-D DDR5  LGA 1700 micro-ATX motherboard   PCIe 4.0  M.2 slot  Realtek 1 Gb Ethernet  HDMI     VGA  USB 3.2 Gen 1  SATA 6 Gbps  COM port  RGB header - 4350
Inserting product: MSI MAG X870 Tomahawk WiFi Gaming Motherboard  AMD Ryzen 9000 8000 7000 Series Processors  AM5  DDR5  PCIe 5.0  M.2 Gen5  SATA 6Gb s  USB 40Gbps  HDMI DP  Wi-Fi 7  Bluetooth 5.4  5Gbps LAN  ATX  - 19499
Inserting product: MSI PRO X870-P WIFI Motherboard  ATX - Supports AMD Ryzen 9000   8000   7000 Processors  AM5 - 60A SPS VRM  DDR5 Memory Boost 8200 MT s  OC   PCIe 5.0 x16   4.0 x16  M.2 Gen5  Wi-Fi 7  5G LAN - 15499
Inserting product: MSI MAG Z890 Tomahawk WiFi Gaming Motherboard  Supports Core Ultra Series 2 Intel Processors  LGA 1851  DDR5  PCIe 5.0  

In [1]:
%pip install psycopg2
%pip install requests
%pip install beautifulsoup4




   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.2 MB 326.8 kB/s eta 0:00:04
   --- ------------------------------------ 0.1/1.2 MB 585.1 kB/s eta 0:00:02
   ----- ---------------------------------- 0.2/1.2 MB 821.4 kB/s eta 0:00:02
   ------- -------------------------------- 0.2/1.2 MB 919.0 kB/s eta 0:00:02
   -------- ------------------------------- 0.3/1.2 MB 983.0 kB/s eta 0:00:01
   ------------ --------------------------- 0.4/1.2 MB 1.1 MB/s eta 0:00:01
   --------------- ------------------------ 0.4/1.2 MB 1.1 MB/s eta 0:00:01
   ----------------- ---------------------- 0.5/1.2 MB 1.2 MB/s eta 0:00:01
   ----------------- ---------------------- 0.5/1.2 MB 1.2 MB/s eta 0:00:01
   ----------------- ---------------------- 0.5/1.2 MB 1.2 MB/s eta 0:00:01
   ----------------- -


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
   ---------------------------------------- 0.0/164.9 kB ? eta -:--:--
   --------- ----------------------------- 41.0/164.9 kB 960.0 kB/s eta 0:00:01
   ----------------------- -------------- 102.4/164.9 kB 980.4 kB/s eta 0:00:01
   ----------------------------- ---------- 122.9/164.9 kB 1.0 MB/s eta 0:00:01
   -------------------------------------  163.8/164.9 kB 893.0 kB/s eta 0:00:01
   -------------------------------------- 164.9/164.9 kB 822.8 kB/s eta 0:00:00
   ---------------------------------------- 0.0/102.4 kB ? eta -:--:--
   --------------- ----------------------- 41.0/102.4 kB 991.0 kB/s eta 0:00:01
   ---------------------------------------- 102.4/102.4 kB 1.2 MB/s eta 0:00:00
Using cached idna-3.10-py3-none-any.whl (70 kB)
   ---------------------------------------- 0.0/128.4 kB ? eta -:--:--


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/147.9 kB ? eta -:--:--
   -- ------------------------------------- 10.2/147.9 kB ? eta -:--:--
   -------- ------------------------------ 30.7/147.9 kB 435.7 kB/s eta 0:00:01
   ------------------ -------------------- 71.7/147.9 kB 563.7 kB/s eta 0:00:01
   ------------------------------------ - 143.4/147.9 kB 853.3 kB/s eta 0:00:01
   -------------------------------------- 147.9/147.9 kB 882.6 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
